In [23]:
# Yerbol Aussat (SID: 20698564)
# CS698, Assignment 3
# Logistic Regression (One vs All)
import numpy as np

In [24]:
# unpickle the data
def unpickle(file):
    import cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo)
    return dict

In [25]:
#import training data
batch1_train = unpickle("cifar-10-batches-py/data_batch_1")
trn_data = batch1_train['data']
trn_labels = np.array(batch1_train['labels'])

In [26]:
# import testing data
batch_test = unpickle("cifar-10-batches-py/test_batch")
test_data = batch_test['data']
test_labels = np.array(batch_test['labels'])

In [27]:
def sigmoid(x, w):                                                        
    z = np.dot(x, w)
    return 1.0 / (1.0 + np.exp(-z)) 

In [28]:
def gradient(x, y, w, reg_const, cl):   
    # indices of positive and negative labels
    y_pos = np.where(y == cl) # locations of y = +1
    y_neg = np.where(y != cl) # locations of y = -1
    
    # number of positive and negative smaples
    n_pos = len(y_pos[0])
    n_neg = len(y_neg[0])
    p = sigmoid(x, w)
    # gradient:
    g = 1.0/n_pos * np.dot(x[y_pos].T, (p[y_pos] - np.ones((n_pos, 1)) )) + 1.0/n_neg * np.dot(x[y_neg].T, (p[y_neg])) + 2.0 *reg_const * w
    return g
                                       

In [29]:
def hessian(x, y, w, reg_const, cl):  
    # indices of positive and negative labels
    y_pos = np.where(y == cl) # locations of y = +1
    y_neg = np.where(y != cl) # locations of y = -1
    
    # number of positive and negative smaples
    n_pos = len(y_pos[0])
    n_neg = len(y_neg[0])
    p = sigmoid(x, w)
        
    # Hessian:
    H = 1.0/n_pos * x[y_pos].T.dot(np.diagflat(p[y_pos])).dot(np.diagflat(np.ones((n_pos, 1)) - p[y_pos])).dot(x[y_pos]) + 1.0/n_neg * x[y_neg].T.dot(np.diagflat(p[y_neg])).dot(np.diagflat(np.ones((n_neg, 1)) - p[y_neg])).dot(x[y_neg]) + 2.0 * reg_const * np.identity(len(w))
    return H
    

In [30]:
from copy import deepcopy

def Newton(X_train, y_train, reg_const, cl):
    w = np.zeros((3072, 1))
    max_pass = 50
    
    iteration = 0
    
    while True:
        print "    iteration", iteration, ": ",
        
        w_prev = deepcopy(w)
        g = gradient(X_train, y_train, w, reg_const, cl)
        H = hessian(X_train, y_train, w, reg_const, cl)
        w = w_prev - np.linalg.solve(H, g)

        print np.linalg.norm(w - w_prev)
        iteration += 1 
        
        if (np.linalg.norm(w - w_prev) < 10**-4) or (iteration >= 50):
            return w           
    return w

In [31]:
# Function calculates percent error of  logistic regression algorithm
def percent_error(trn_data, trn_labels, test_data, test_labels, lambdas):
    
    for i in range(len(lambdas)):   
        reg_const = lambdas[i]
        w_vectors = []
        print "\nONE VS ALL: lambda =", reg_const
        
        # Training kernel logistic regression algorithm for the value of lambda
        print "Training logistic regression for lambda =", reg_const
        for i in range(10):
            print "\n  CLASS", i
            w_i = Newton(trn_data, trn_labels, reg_const, i)
            w_vectors.append(w_i)
        
        # Testing kernel logistic regression algorithm for lambda = 1
        print "Testing logistic regression for lambda =", reg_const
        incorrect = 0
        for sample_i in range(len(test_labels)):
            x = test_data[sample_i]
            p = []
            for i in range(10):
                w0 = w_vectors[i]
                p0 = np.dot(w0.T, x) #p0 = sigmoid(x, w0)
                p.append(p0)
            prediction = np.argmax(p)
            if prediction != test_labels[sample_i]:
                incorrect+=1
                
        print "\nOne vs All: lambda =", reg_const, ":"
        print "  Num Incorrectly Classified:", incorrect
        print "  Percent Error:", 1.0*incorrect / len(test_labels)
        print "\n", '*'*40, '\n'

In [32]:
# Finding percent error for lambdas = 0.1, 1, 10
percent_error(trn_data, trn_labels, test_data, test_labels, [0.1])


ONE VS ALL: lambda = 0.1
Training logistic regression for lambda = 0.1

  CLASS 0
    iteration 0 :  0.439136438427
    iteration 1 :  0.223073027283
    iteration 2 :  0.186901394459
    iteration 3 :  0.124386092956
    iteration 4 :  0.048775499788
    iteration 5 :  0.00675425913587
    iteration 6 :  0.000156232995119
    iteration 7 :  1.47933395375e-07

  CLASS 1
    iteration 0 :  0.427668293617
    iteration 1 :  0.213156088216
    iteration 2 :  0.192667945279
    iteration 3 :  0.16610295977
    iteration 4 :  0.124605522531
    iteration 5 :  0.0584718366399
    iteration 6 :  0.0121113528642
    iteration 7 :  0.000590342173505
    iteration 8 :  2.29669420029e-06

  CLASS 2
    iteration 0 :  0.50929105672
    iteration 1 :  0.239085348763
    iteration 2 :  0.174251896775
    iteration 3 :  0.0857165910648
    iteration 4 :  0.0202821672256
    iteration 5 :  0.00114047164789
    iteration 6 :  6.8010504296e-06

  CLASS 3
    iteration 0 :  0.496684421841
    iteration 

In [33]:
percent_error(trn_data, trn_labels, test_data, test_labels, [1])


ONE VS ALL: lambda = 1
Training logistic regression for lambda = 1

  CLASS 0
    iteration 0 :  0.165769640632
    iteration 1 :  0.072899244647
    iteration 2 :  0.0494241717706
    iteration 3 :  0.0198616363915
    iteration 4 :  0.0027842101685
    iteration 5 :  0.00010835387399
    iteration 6 :  6.5880301158e-07

  CLASS 1
    iteration 0 :  0.170312590433
    iteration 1 :  0.0748954392698
    iteration 2 :  0.058050682675
    iteration 3 :  0.0364628284309
    iteration 4 :  0.0119011783184
    iteration 5 :  0.00115984011274
    iteration 6 :  1.3330002746e-05

  CLASS 2
    iteration 0 :  0.189793499129
    iteration 1 :  0.0713307418017
    iteration 2 :  0.0352022064359
    iteration 3 :  0.00885834971374
    iteration 4 :  0.000623690328779
    iteration 5 :  6.07445883654e-06

  CLASS 3
    iteration 0 :  0.188441856921
    iteration 1 :  0.0697574425103
    iteration 2 :  0.0380758620547
    iteration 3 :  0.0106710218549
    iteration 4 :  0.000785266686965
    iter

In [34]:
percent_error(trn_data, trn_labels, test_data, test_labels, [10])


ONE VS ALL: lambda = 10
Training logistic regression for lambda = 10

  CLASS 0
    iteration 0 :  0.0586425421105
    iteration 1 :  0.0209560505942
    iteration 2 :  0.00932956574315
    iteration 3 :  0.00172173630652
    iteration 4 :  8.62099406288e-05

  CLASS 1
    iteration 0 :  0.0650190678603
    iteration 1 :  0.0242587131961
    iteration 2 :  0.0139036903026
    iteration 3 :  0.00430091789549
    iteration 4 :  0.000371497396157
    iteration 5 :  3.06095024211e-06

  CLASS 2
    iteration 0 :  0.0650827602944
    iteration 1 :  0.018788991749
    iteration 2 :  0.00579337462738
    iteration 3 :  0.000631810277774
    iteration 4 :  8.86797598791e-06

  CLASS 3
    iteration 0 :  0.0651187883684
    iteration 1 :  0.0182029610259
    iteration 2 :  0.00693769634869
    iteration 3 :  0.000707636597148
    iteration 4 :  1.01517569892e-05

  CLASS 4
    iteration 0 :  0.0652439189175
    iteration 1 :  0.0190953742549
    iteration 2 :  0.00706770812845
    iteration 3 